In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
movies_df = pd.read_csv('/content/drive/MyDrive/tmdb79_movies.csv')

In [ ]:
movies_df.head(5)

,movieCd,movieNm,movieNmEn,showTm,prdtYear,openDt,genres,directors,actors,poster_url,keywords,vote_average,vote_count
0,20233217,더 킬러스,The Killers,118.0,2023.0,20241023,"드라마, 액션, 스릴러","이명세, 장항준, 노덕, 김종관","심은경, 연우진, 홍사빈",https://image.tmdb.org/t/p/w500/dB6Krk806zeqd0...,"husband wife relationship, based on novel or b...",7.428,3607.0
1,20246777,더 폴: 디렉터스 컷,The Fall,119.0,2024.0,20241225,"드라마, 판타지",타셈 싱,"리 페이스, 카틴카 언타루",https://image.tmdb.org/t/p/w500/g3RKh7Gp2lDUnX...,"suicide attempt, stuntman, remake, morphine, h...",7.598,1366.0
2,20244912,아노라,Anora,139.0,2024.0,20241106,"드라마, 코미디, 멜로/로맨스",션 베이커,"미키 매디슨, 마크 아이델스테인, 유리 보리소프",https://image.tmdb.org/t/p/w500/7MrgIUeq0DD2iF...,"new year's eve, new york city, marriage, board...",7.045,2148.0
3,20244850,시빌 워: 분열의 시대,Civil War,109.0,2024.0,20241231,"액션, 전쟁, 드라마",알렉스 가랜드,"커스틴 던스트, 케일리 스패니, 와그너 모라",https://image.tmdb.org/t/p/w500/sh7Rg8Er3tFcN9...,"sniper, new york city, race against time, wash...",6.865,3560.0
4,20244762,서브스턴스,THE SUBSTANCE,141.0,2024.0,20241211,스릴러,코랄리 파르쟈,"데미 무어, 마가렛 퀄리, 데니스 퀘이드",https://image.tmdb.org/t/p/w500/evdF1vmLzuzH8E...,"new year's eve, capitalism, black market, iden...",7.100,4572.0


In [ ]:
# 중요한 column만 선택
selected_columns = ['movieCd', 'movieNm', 'genres', 'keywords']
movies_selected_df = movies_df[selected_columns]

In [ ]:
movies_selected_df.head(5)

,movieCd,movieNm,genres,keywords
0,20233217,더 킬러스,"드라마, 액션, 스릴러","husband wife relationship, based on novel or b..."
1,20246777,더 폴: 디렉터스 컷,"드라마, 판타지","suicide attempt, stuntman, remake, morphine, h..."
2,20244912,아노라,"드라마, 코미디, 멜로/로맨스","new year's eve, new york city, marriage, board..."
3,20244850,시빌 워: 분열의 시대,"액션, 전쟁, 드라마","sniper, new york city, race against time, wash..."
4,20244762,서브스턴스,스릴러,"new year's eve, capitalism, black market, iden..."


In [ ]:
# TF-IDF 벡터화 객체 (한글+영어 혼합, 불용어 직접 관리 가능)
tfidf_vectorizer = TfidfVectorizer(
    token_pattern=r"(?u)\b\w+\b",  # 한글+영어 혼합 토큰 허용
)

# 장르/키워드 합치기 (문자형 변환 + 공백 구분)
movies_df['content'] = movies_df['genres'].astype(str) + ' ' + movies_df['keywords'].astype(str)

# TF-IDF 행렬 생성
tfidf_matrix = tfidf_vectorizer.fit_transform(movies_df['content'])

In [ ]:
# 코사인 유사도 계산
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 콘텐츠 기반 필터링 함수 정의

In [ ]:
def content_based_filtering(title, cosine_sim=cosine_sim):
    # 1. 입력받은 영화 제목(title)에 해당하는 인덱스 찾기
    idx = movies_df[movies_df['movieNm'] == title].index[0]

    # 2. 해당 영화와 다른 영화들 간의 유사도 리스트 추출
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 3. 유사도 점수 기준으로 내림차순 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 4. 자기 자신을 제외한 상위 3개 추출
    sim_scores = sim_scores[1:4]

    # 5. 추천 대상 영화의 인덱스 추출
    movie_indices = [i[0] for i in sim_scores]

    # 6. 추천 결과 반환 (영화 제목들)
    return movies_df['movieNm'].iloc[movie_indices]

In [ ]:
# 테스트
result = content_based_filtering('더 킬러스')
print(result)

40    존 오브 인터레스트
76        추락의 해부
6          더 크로우
Name: movieNm, dtype: object


### .apply()로 각 영화 제목에 대해 추천 3개를 추출

추천 결과는 리스트 → 문자열로 변환 (쉼표 구분)

예외 발생 시 '추천 없음' 반환

In [ ]:
# 추천 결과를 문자열로 저장하는 함수
def get_recommendations(title):
    try:
        result = content_based_filtering(title)
        return ', '.join(result.values)
    except:
        return '추천 없음'

# 전체 영화에 대해 recommend 열 생성
movies_df['recommend'] = movies_df['movieNm'].apply(get_recommendations)

In [ ]:
print(movies_df[['movieNm', 'recommend']].head())

        movieNm                    recommend
0         더 킬러스    존 오브 인터레스트, 추락의 해부, 더 크로우
1   더 폴: 디렉터스 컷     탐정 말로, 스턴트맨, 플라이 미 투 더 문
2           아노라       룸 넥스트 도어, 서브스턴스, 어프렌티스
3  시빌 워: 분열의 시대  전장의 크리스마스, 더 커버넌트, 룸 넥스트 도어
4         서브스턴스            탐정 말로, 아노라, 스마일 2


In [ ]:
movies_df.to_csv("추천결과.csv", index=False, encoding='utf-8-sig')

# ------------------------------------

# 추천 이유

In [ ]:
def recommend_and_reason_sentence(title, idx_map, df, sim_matrix):
    try:
        idx = idx_map[title]
    except:
        return '', ''

    sim_scores = list(enumerate(sim_matrix[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:4]

    recommendations = []
    reason_sentences = []

    input_genres = set(df.loc[idx, 'genres'].replace(' ', '').split(','))
    input_keywords = set(str(df.loc[idx, 'keywords']).lower().split(', '))

    for i, _ in sim_scores:
        row = df.iloc[i]
        rec_title = row['movieNm']
        rec_genres = set(row['genres'].replace(' ', '').split(','))
        rec_keywords = set(str(row['keywords']).lower().split(', '))

        # 교집합
        genre_overlap = input_genres & rec_genres
        keyword_overlap = input_keywords & rec_keywords

        # 자연어 형태의 추천 이유 문장 만들기
        reason_parts = []
        if genre_overlap:
            genre_text = '’, ‘'.join(genre_overlap)
            reason_parts.append(f"‘{genre_text}’ 장르를 공유하며")
        if keyword_overlap:
            keyword_text = '’, ‘'.join(keyword_overlap)
            reason_parts.append(f"키워드 ‘{keyword_text}’가 유사합니다")

        if reason_parts:
            reason_sentence = f"{rec_title}는 이 영화와 " + ', '.join(reason_parts) + '.'
        else:
            reason_sentence = f"{rec_title}는 유사한 영화입니다."

        recommendations.append(rec_title)
        reason_sentences.append(reason_sentence)

    return ', '.join(recommendations), ' '.join(reason_sentences)

In [ ]:
title_to_idx = pd.Series(movies_df.index, index=movies_df['movieNm']).to_dict()

In [ ]:
recommendations = []
reasons = []

for title in movies_df['movieNm']:
    recs, reason_text = recommend_and_reason_sentence(title, title_to_idx, movies_df, cosine_sim)
    recommendations.append(recs)
    reasons.append(reason_text)

movies_df['recommend'] = recommendations
movies_df['reason'] = reasons

In [ ]:
print(movies_df[['movieNm', 'recommend', 'reason']].head(1))

  movieNm                  recommend  \
0   더 킬러스  존 오브 인터레스트, 추락의 해부, 더 크로우   

                                              reason  
0  존 오브 인터레스트는 이 영화와 ‘드라마’ 장르를 공유하며, 키워드 ‘husband...  


In [ ]:
movies_df.to_csv('movie_with_recommendation_and_reason.csv', index=False, encoding='utf-8-sig')